In [62]:
import spacy
from spacy.lang.fr.stop_words import STOP_WORDS
from spacy_lefff import LefffLemmatizer, POSTagger
import json

import logging

In [63]:
def load_json_data(filename: str) -> dict:
    """
    Load the content of the json file as a dict.
    """
    f = open(filename, 'r')
    file_content = json.load(f)
    f.close()
    return file_content


def get_raw_training_data(path: str = './training_set.json'):
    """
    Load the raw training set.
    """
    return load_json_data(path)


def get_raw_testing_data(path: str = './testing_set.json'):
    """
    Load the raw testing set.
    """
    return load_json_data(path)

In [64]:
raw_training_data = get_raw_training_data()

In [124]:
X = [v['sentence'] for v in raw_training_data]
y = [v['intent'] for v in raw_training_data]

In [109]:
nlp = spacy.load('fr_core_news_sm')

logging.getLogger('spacy_lefff').setLevel(logging.WARNING)

pos = POSTagger()
french_lemmatizer = LefffLemmatizer(after_melt=True)
nlp.add_pipe(pos, name='pos', after='parser')
nlp.add_pipe(french_lemmatizer, name='lefff', after='parser')

In [127]:
def pre_process(nlp, X):
    new_X = []
    for sentence in X:
        doc = nlp(sentence)
        lemmas = []
        for token in doc:                                # Tokenize
            if not token.is_stop and not token.is_punct: # Remove stop words and punctuation
                lemmas.append(token)                     # Because of pipes added, each token has it's lemma
        new_X.append(lemmas)
    return new_X

X = pre_process(nlp, X)

In [153]:
for token in X[6]:
    print(token.text, token.lemma_)

pouvez pouvoir
réserver réserver
09h 09h
oui oui
